In [33]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#import deepLIFT stuff
scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR to point to the deeplift code (WITHIN the deeplift repo)");
sys.path.insert(0,scriptsDir);
#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

import keras_conversion as kc

In [34]:
#Load the keras model, make sure you normalise the weights
#of the first convolutional layer to be mean-centered at each position.
model_weights = "hybrid_CTCF_Helas3_vsDNase_1000bp_conv3LowFiltTopHap.hdf5"
model_yaml = "hybrid_CTCF_Helas3_vsDNase_1000bp_conv3LowFiltTopHap_architecture.json"
reload(kc)
keras_model = kc.load_keras_model(model_weights, model_yaml, normalise_conv_for_one_hot_encoded_input=True)

In [35]:
keras_model.layers[0].get_config()

{'W_constraint': None,
 'W_learning_rate_multiplier': None,
 'W_regularizer': {'l1': 0.0,
  'l2': 0.0,
  'name': 'WeightRegularizer',
  'zr': 0.0},
 'activation': 'linear',
 'activity_regularizer': None,
 'b_constraint': None,
 'b_learning_rate_multiplier': None,
 'b_regularizer': None,
 'border_mode': 'valid',
 'cache_enabled': True,
 'custom_name': 'convolution2d',
 'dim_ordering': 'th',
 'init': 'he_normal',
 'input_shape': (1, 4, 1000),
 'name': 'Convolution2D',
 'nb_col': 15,
 'nb_filter': 60,
 'nb_row': 4,
 'subsample': (1, 1),
 'trainable': True}

In [36]:
keras_model.layers[-1].get_config()

{'activation': 'sigmoid',
 'cache_enabled': True,
 'custom_name': 'activation',
 'name': 'Activation',
 'trainable': True}

In [38]:
from blobs import MxtsMode
import theano
deeplift_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.DeepLIFT)
deeplift_contribs_func = deeplift_model.get_target_contribs_func(input_layer_idx=0)

In [ ]:
deeplift_model.get_layers()[0].update_mxts()

In [31]:
theano.function([deeplift_model.get_layers()[0].get_activation_vars()],
                deeplift_model.get_layers()[1].get_target_contrib_vars())